In [1]:
import json
with open('/home/user/data/DataCompDR-12M-bf16/00000150/0a0afe817f1283fb6f23371d116a0d09.paug.json','r') as f:
    dic = json.load(f)
    

In [2]:
print(len(dic['param_aug']))

30


In [8]:
import pickle
import gzip
import torch

with gzip.open('/home/user/data/DataCompDR-12M-bf16/00000150/0a0f57ee3267795252a71834ae4e8a48.pth.gz', 'rb') as f:
    tensor = torch.load(f)

In [9]:
print(len(tensor['text_emb']))

7


In [16]:
tensor['text_emb']

tensor([[ 0.0439, -0.0200, -0.0645,  ...,  0.0723,  0.0071,  0.0087],
        [ 0.0439, -0.0200, -0.0645,  ...,  0.0723,  0.0071,  0.0087],
        [ 0.0031, -0.0339, -0.0136,  ...,  0.0132,  0.0854, -0.0225],
        ...,
        [ 0.0275,  0.0447, -0.0084,  ...,  0.0175,  0.0508,  0.0325],
        [ 0.0281, -0.0233, -0.0549,  ...,  0.0281,  0.1250,  0.0605],
        [ 0.0129, -0.0168,  0.0084,  ...,  0.0183,  0.0162,  0.0183]],
       dtype=torch.bfloat16)

In [11]:
tensor['text_emb'][0]

tensor([ 0.0439, -0.0200, -0.0645,  ...,  0.0723,  0.0071,  0.0087],
       dtype=torch.bfloat16)

In [12]:
tensor['text_emb'][1]

tensor([ 0.0439, -0.0200, -0.0645,  ...,  0.0723,  0.0071,  0.0087],
       dtype=torch.bfloat16)

In [13]:
tensor['text_emb'][2]

tensor([ 0.0031, -0.0339, -0.0136,  ...,  0.0132,  0.0854, -0.0225],
       dtype=torch.bfloat16)

In [14]:
tensor['text_emb'][0] == tensor['text_emb'][1]

tensor([True, True, True,  ..., True, True, True])

In [15]:
for i in (tensor['text_emb'][0] == tensor['text_emb'][1]):
    if i == True:
        continue
    else:
        print(False)
        break

In [17]:
torch.sum(tensor['text_emb'][0])

tensor(1.5469, dtype=torch.bfloat16)

In [23]:
torch.norm(torch.nn.functional.normalize(tensor['text_emb'][0],dim=-1))

tensor(1., dtype=torch.bfloat16)